In [295]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [296]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [297]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs',  'results.gdb', 'TAZ_Grouping_Analysis.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [298]:
roads = r'.\Inputs\Utah_Roads.gdb\Roads'
buildings = r'.\Inputs\remm_base_year.gdb\buildings_pts'
se_2019 = r'.\inputs\se_2019.shp'
hui_2019 = r'.\inputs\wfrc_mag_housing_inventory_2020_20220209.gdb\housing_inventory_2020_20220209_pts'
taz_nbhd = r'.\Inputs\TAZ_Neighborhoods.shp'
# taz_900 = pd.DataFrame.spatial.from_featureclass(r'.\Inputs\TAZ9.shp')

In [299]:
# dissolve taz into neighborhood shapes
nbhd = arcpy.management.Dissolve(taz_nbhd, os.path.join(gdb, '_00_nbhd'), 'Category', "ACRES SUM;DEVACRES SUM",  "SINGLE_PART", "DISSOLVE_LINES")
nbhd_lines = arcpy.management.PolygonToLine(nbhd, os.path.join(gdb, '_01_nbhd_lines'), "IGNORE_NEIGHBORS")
nbhd_buff = arcpy.analysis.Buffer(nbhd_lines, os.path.join(gdb, '_02_nbhd_bnd_buff'), "20 Meters", "FULL", "ROUND")
nbhd_erase = arcpy.analysis.Erase(nbhd, nbhd_buff, os.path.join(gdb, '_03_nbhd_erase'))

nbhd_df = pd.DataFrame.spatial.from_featureclass(nbhd[0])
nbhd_df.rename({'SUM_ACRES':'ACRES', 'SUM_DEVACRES':'DEVACRES'}, inplace=True, axis=1)
del nbhd_df['OBJECTID']

### Run summarize_within.py code in arcgis pro

In [301]:
# arterial = ''' CARTOCODE IN ('8 - 8 Major Local Roads, Paved', '2', '3', '4', '5') '''
# local = ''' CARTOCODE IN ('8 - 8 Major Local Roads, Paved', '11', '10', '9', '8') '''
# collector =  ''' CARTOCODE IN ('8 - 8 Major Local Roads, Paved', '7') '''
# roads_layer = arcpy.MakeFeatureLayer_management("Roads", 'roads_lyr')

# #=========================
# # inside (100% of length)
# #=========================

# # arterial
# arcpy.SelectLayerByAttribute_management(roads_layer, 'NEW_SELECTION', arterial)
# inside_sw_arterial = arcpy.analysis.SummarizeWithin('_03_nbhd_erase', roads_layer, os.path.join(gdb, '_04_inside_sw_arterial'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')

# # local
# arcpy.SelectLayerByAttribute_management(roads_layer, 'NEW_SELECTION', local)
# inside_sw_local = arcpy.analysis.SummarizeWithin('_03_nbhd_erase', roads_layer, os.path.join(gdb, '_04_inside_sw_local'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')

# # collector
# arcpy.SelectLayerByAttribute_management(roads_layer, 'NEW_SELECTION', collector)
# inside_sw_collector = arcpy.analysis.SummarizeWithin('_03_nbhd_erase', roads_layer, os.path.join(gdb, '_04_inside_sw_collector'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')

# # all
# arcpy.SelectLayerByAttribute_management(roads_layer, 'CLEAR_SELECTION')
# inside_sw_all = arcpy.analysis.SummarizeWithin('_03_nbhd_erase', roads_layer, os.path.join(gdb, '_04_inside_sw_all'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')

# #===========================
# # boundary (50% of length)
# #===========================

# # arterial
# arcpy.SelectLayerByAttribute_management(roads_layer, 'NEW_SELECTION', arterial)
# boundary_sw_arterial = arcpy.analysis.SummarizeWithin('_02_nbhd_bnd_buff', roads_layer, os.path.join(gdb, '_05_boundary_sw_arterial'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')
# # local
# arcpy.SelectLayerByAttribute_management(roads_layer, 'NEW_SELECTION', local)
# boundary_sw_local = arcpy.analysis.SummarizeWithin('_02_nbhd_bnd_buff', roads_layer, os.path.join(gdb, '_05_boundary_sw_local'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')
# # collector
# arcpy.SelectLayerByAttribute_management(roads_layer, 'NEW_SELECTION', collector)
# boundary_sw_collector = arcpy.analysis.SummarizeWithin('_02_nbhd_bnd_buff', roads_layer, os.path.join(gdb, '_05_boundary_sw_collector'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')

# # all
# arcpy.SelectLayerByAttribute_management(roads_layer, 'CLEAR_SELECTION')
# boundary_sw_all = arcpy.analysis.SummarizeWithin('_02_nbhd_bnd_buff', roads_layer, os.path.join(gdb, '_05_boundary_sw_all'), "KEEP_ALL", "Miles Sum", "ADD_SHAPE_SUM", 'MILES')

### Roads

In [302]:
boundary_sw_arterial = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_05_boundary_sw_arterial'))[['Category', 'sum_Length_MILES']].copy()
boundary_sw_arterial.rename({'sum_Length_MILES': 'Miles_A_B'}, axis=1, inplace=True)
boundary_sw_arterial['Miles_A_B'] = boundary_sw_arterial['Miles_A_B'] / 2

boundary_sw_local = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_05_boundary_sw_local'))[['Category', 'sum_Length_MILES']].copy()
boundary_sw_local.rename({'sum_Length_MILES': 'Miles_L_B'}, axis=1, inplace=True)
boundary_sw_local['Miles_L_B'] = boundary_sw_local['Miles_L_B'] / 2

boundary_sw_collector = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_05_boundary_sw_collector'))[['Category', 'sum_Length_MILES']].copy()
boundary_sw_collector.rename({'sum_Length_MILES': 'Miles_C_B'}, axis=1, inplace=True)
boundary_sw_collector['Miles_C_B'] = boundary_sw_collector['Miles_C_B'] / 2

boundary_sw_all = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_05_boundary_sw_all'))[['Category', 'sum_Length_MILES']].copy()
boundary_sw_all.rename({'sum_Length_MILES': 'Miles_ALL_B'}, axis=1, inplace=True)
boundary_sw_all['Miles_ALL_B'] = boundary_sw_all['Miles_ALL_B'] / 2

inside_sw_arterial = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_04_inside_sw_arterial')) [['Category', 'sum_Length_MILES']].copy()
inside_sw_arterial.rename({'sum_Length_MILES': 'Miles_A_I'}, axis=1, inplace=True)

inside_sw_local = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_04_inside_sw_local'))[['Category', 'sum_Length_MILES']].copy()
inside_sw_local.rename({'sum_Length_MILES': 'Miles_L_I'}, axis=1, inplace=True)

inside_sw_collector = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_04_inside_sw_collector'))[['Category', 'sum_Length_MILES']].copy()
inside_sw_collector.rename({'sum_Length_MILES': 'Miles_C_I'}, axis=1, inplace=True)

inside_sw_all = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_04_inside_sw_all'))[['Category', 'sum_Length_MILES']].copy()
inside_sw_all.rename({'sum_Length_MILES': 'Miles_ALL_I'}, axis=1, inplace=True)

In [303]:
nbhd_df_merge = (nbhd_df.merge(boundary_sw_arterial, on='Category', how='left')
                        .merge(boundary_sw_local, on='Category', how='left')
                        .merge(boundary_sw_collector, on='Category', how='left')
                        .merge(boundary_sw_all, on='Category', how='left')
                        .merge(inside_sw_arterial, on='Category', how='left')
                        .merge(inside_sw_local, on='Category', how='left')
                        .merge(inside_sw_collector, on='Category', how='left')
                        .merge(inside_sw_all, on='Category', how='left')

)

nbhd_df_merge['MILES_ARTERIAL'] = nbhd_df_merge['Miles_A_B'] + nbhd_df_merge['Miles_A_I']
nbhd_df_merge['MILES_COLLECTOR'] = nbhd_df_merge['Miles_C_B'] + nbhd_df_merge['Miles_C_I']
nbhd_df_merge['MILES_LOCAL'] = nbhd_df_merge['Miles_L_B'] + nbhd_df_merge['Miles_L_I']
nbhd_df_merge['MILES_ANY_ROAD'] = nbhd_df_merge['Miles_ALL_B'] + nbhd_df_merge['Miles_ALL_I']

nbhd_df_merge = nbhd_df_merge[['Category', 'SHAPE', 'DEVACRES', 'ACRES', 'MILES_ARTERIAL', 'MILES_COLLECTOR', 'MILES_LOCAL', 'MILES_ANY_ROAD']].copy()

nbhd_df_merge.head(5)

,Category,SHAPE,DEVACRES,ACRES,MILES_ARTERIAL,MILES_COLLECTOR,MILES_LOCAL,MILES_ANY_ROAD
0,9th and 9th,"{'rings': [[[426491.5, 4512071.1], [426549.5, ...",391.556647,391.556647,0.51901,0.0,12.812107,13.331117
1,"Avenues, Lower","{'rings': [[[425840.5, 4513912.4], [425916.200...",60.511321,60.511321,0.00000,0.0,2.408423,2.408423
2,"Avenues, Upper","{'rings': [[[425951.53000000026, 4514917.67], ...",78.696582,78.696582,0.00000,0.0,3.233425,3.233425
3,"Daybreak, High Den","{'rings': [[[415038.4000000004, 4488766.800000...",210.999718,210.999718,0.00000,0.0,10.890334,10.890334
4,"Daybreak, Low Den","{'rings': [[[415799.4000000004, 4490379.4], [4...",391.942869,391.942869,0.00000,0.0,18.340754,18.340754


### Housing Unit Inventory

In [304]:
#========================
# Housing unit inventory
#========================

#queries
sf = ''' SUBTYPE = 'single_family' '''
th = ''' SUBTYPE = 'townhome' '''
mf = ''' TYPE = 'multi_family' And SUBTYPE <> 'townhome' '''
hui_layer = arcpy.MakeFeatureLayer_management(hui_2019, 'hui_lyr')

#===============
# single family
#===============

# number of sf units
arcpy.SelectLayerByAttribute_management(hui_layer, 'NEW_SELECTION', sf)

target_features = nbhd 
join_features = hui_layer
output_features = os.path.join(gdb, "_06_hui_sf")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum units
fieldindex = fieldmappings.findFieldMapIndex('UNIT_COUNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average sqft of units
fieldindex = fieldmappings.findFieldMapIndex('SQFT_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average size of  parcels
fieldindex = fieldmappings.findFieldMapIndex('ACRES_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

sf_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

sf_sj_df = pd.DataFrame.spatial.from_featureclass(sf_sj[0])

#===============
# townhome
#===============

# number of units
arcpy.SelectLayerByAttribute_management(hui_layer, 'NEW_SELECTION', th)

target_features = nbhd 
join_features = hui_layer
output_features = os.path.join(gdb, "_06_hui_th")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum units
fieldindex = fieldmappings.findFieldMapIndex('UNIT_COUNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average sqft of units
fieldindex = fieldmappings.findFieldMapIndex('SQFT_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

th_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

th_sj_df = pd.DataFrame.spatial.from_featureclass(th_sj[0])

#===============
# multi family
#===============

# number of units
arcpy.SelectLayerByAttribute_management(hui_layer, 'NEW_SELECTION', mf)

target_features = nbhd 
join_features = hui_layer
output_features = os.path.join(gdb, "_06_hui_mf")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum units
fieldindex = fieldmappings.findFieldMapIndex('UNIT_COUNT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average sqft of units
fieldindex = fieldmappings.findFieldMapIndex('SQFT_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

mf_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

mf_sj_df = pd.DataFrame.spatial.from_featureclass(mf_sj[0])

In [305]:
sf_sj_df = sf_sj_df[['Category', 'UNIT_COUNT', 'SQFT_PER_UNIT', 'ACRES_PER_UNIT']].copy()
sf_sj_df.rename({'UNIT_COUNT': 'SF_UNITS', 'SQFT_PER_UNIT':'AVG_SF_FT2', 'ACRES_PER_UNIT':'AVG_SF_ACRES'}, axis=1, inplace=True)

th_sj_df = th_sj_df[['Category', 'UNIT_COUNT', 'SQFT_PER_UNIT']].copy()
th_sj_df.rename({'UNIT_COUNT': 'TH_UNITS', 'SQFT_PER_UNIT':'AVG_TH_FT2'}, axis=1, inplace=True)

mf_sj_df = mf_sj_df[['Category', 'UNIT_COUNT', 'SQFT_PER_UNIT']].copy()
mf_sj_df.rename({'UNIT_COUNT': 'MF_UNITS', 'SQFT_PER_UNIT':'AVG_MF_FT2'}, axis=1, inplace=True)

In [306]:
# merge the new data
nbhd_df_merge2 = (nbhd_df_merge.merge(sf_sj_df, on='Category', how='left')
                        .merge(th_sj_df, on='Category', how='left')
                        .merge(mf_sj_df, on='Category', how='left')
)

nbhd_df_merge2.head()

,Category,SHAPE,DEVACRES,ACRES,MILES_ARTERIAL,MILES_COLLECTOR,MILES_LOCAL,MILES_ANY_ROAD,SF_UNITS,AVG_SF_FT2,AVG_SF_ACRES,TH_UNITS,AVG_TH_FT2,MF_UNITS,AVG_MF_FT2
0,9th and 9th,"{'rings': [[[426491.5, 4512071.1], [426549.5, ...",391.556647,391.556647,0.51901,0.0,12.812107,13.331117,1275.0,2007.230591,0.136204,16.0,2019.776733,1013.0,1030.005127
1,"Avenues, Lower","{'rings': [[[425840.5, 4513912.4], [425916.200...",60.511321,60.511321,0.00000,0.0,2.408423,2.408423,88.0,2657.136475,0.128182,NaN,NaN,537.0,882.633118
2,"Avenues, Upper","{'rings': [[[425951.53000000026, 4514917.67], ...",78.696582,78.696582,0.00000,0.0,3.233425,3.233425,311.0,2292.061035,0.126881,NaN,NaN,141.0,1211.045898
3,"Daybreak, High Den","{'rings': [[[415038.4000000004, 4488766.800000...",210.999718,210.999718,0.00000,0.0,10.890334,10.890334,277.0,3109.864502,0.153529,294.0,1540.099609,385.0,1231.502441
4,"Daybreak, Low Den","{'rings': [[[415799.4000000004, 4490379.4], [4...",391.942869,391.942869,0.00000,0.0,18.340754,18.340754,1023.0,3491.598145,0.165827,244.0,2448.204834,NaN,NaN


### REMM Buildings

In [307]:
#===================
# REMM buildings
#===================

# buildings
sf = ''' building_type_id = 1 '''
mf = ''' building_type_id = 2 '''
commercial = ''' building_type_id IN (3,4,5,7) '''
other = ''' building_type IN ('Educational', 'Open Space Non-Buildable') Or building_type_id IN (6, 8) '''

b_layer = arcpy.MakeFeatureLayer_management(buildings, 'b_lyr')

#===============
# all parcels
#===============

arcpy.SelectLayerByAttribute_management(b_layer, 'CLEAR_SELECTION')

target_features = nbhd 
join_features = b_layer
output_features = os.path.join(gdb, "_07_remm_all")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum acres
fieldindex = fieldmappings.findFieldMapIndex('ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

remm_all_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

remm_all_sj_df = pd.DataFrame.spatial.from_featureclass(remm_all_sj[0])

#===============
# commercial
#===============

arcpy.SelectLayerByAttribute_management(b_layer, 'NEW_SELECTION', commercial)

target_features = nbhd 
join_features = b_layer
output_features = os.path.join(gdb, "_07_remm_commercial")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum acres
fieldindex = fieldmappings.findFieldMapIndex('ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

remm_com_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

remm_com_sj_df = pd.DataFrame.spatial.from_featureclass(remm_com_sj[0])

#===============
# other
#===============

arcpy.SelectLayerByAttribute_management(b_layer, 'NEW_SELECTION', other)

target_features = nbhd 
join_features = b_layer
output_features = os.path.join(gdb, "_07_remm_commercial")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum acres
fieldindex = fieldmappings.findFieldMapIndex('ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

remm_other_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

remm_other_sj_df = pd.DataFrame.spatial.from_featureclass(remm_other_sj[0])



#===============
# single family
#===============

arcpy.SelectLayerByAttribute_management(b_layer, 'NEW_SELECTION', sf)

target_features = nbhd 
join_features = b_layer
output_features = os.path.join(gdb, "_07_remm_sf")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum acres
fieldindex = fieldmappings.findFieldMapIndex('ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# sum units
fieldindex = fieldmappings.findFieldMapIndex('residential_units')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average sqft of units
fieldindex = fieldmappings.findFieldMapIndex('SQFT_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average size of  parcels
fieldindex = fieldmappings.findFieldMapIndex('ACRES_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

remm_sf_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

remm_sf_sj_df = pd.DataFrame.spatial.from_featureclass(remm_sf_sj[0])

#===============
# multi family
#===============

arcpy.SelectLayerByAttribute_management(b_layer, 'NEW_SELECTION', mf)

target_features = nbhd 
join_features = b_layer
output_features = os.path.join(gdb, "_07_remm_mf")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# sum acres
fieldindex = fieldmappings.findFieldMapIndex('ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# sum units
fieldindex = fieldmappings.findFieldMapIndex('residential_units')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# average sqft of units
fieldindex = fieldmappings.findFieldMapIndex('SQFT_PER_UNIT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

remm_mf_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

remm_mf_sj_df = pd.DataFrame.spatial.from_featureclass(remm_mf_sj[0])

In [308]:
remm_all_sj_df = remm_all_sj_df[['Category', 'ACRES']].copy()
remm_all_sj_df.rename({'ACRES':'TTL_PARCEL_ACRES'}, axis=1, inplace=True)

remm_com_sj_df = remm_com_sj_df[['Category', 'ACRES']].copy()
remm_com_sj_df.rename({'ACRES':'COM_PARCEL_ACRES'}, axis=1, inplace=True)

remm_other_sj_df = remm_other_sj_df[['Category', 'ACRES']].copy()
remm_other_sj_df.rename({'ACRES':'OTHR_PARCEL_ACRES'}, axis=1, inplace=True)

remm_sf_sj_df = remm_sf_sj_df[['Category', 'residential_units', 'SQFT_PER_UNIT', 'ACRES', 'ACRES_PER_UNIT']].copy()
remm_sf_sj_df.rename({'residential_units': 'REMM_SF_UNITS', 'SQFT_PER_UNIT':'REMM_AVG_SF_FT2', 'ACRES_PER_UNIT':'REMM_AVG_SF_ACRES', 'ACRES':'SF_PARCEL_ACRES'}, axis=1, inplace=True)

remm_mf_sj_df = remm_mf_sj_df[['Category', 'residential_units','ACRES', 'SQFT_PER_UNIT']].copy()
remm_mf_sj_df.rename({'residential_units': 'REMM_MF_UNITS', 'SQFT_PER_UNIT':'REMM_AVG_MF_FT2', 'ACRES':'MF_PARCEL_ACRES'}, axis=1, inplace=True)

In [309]:
# merge the new data
nbhd_df_merge3 = (nbhd_df_merge2.merge(remm_all_sj_df, on='Category', how='left')
                                .merge(remm_sf_sj_df, on='Category', how='left')
                                .merge(remm_mf_sj_df, on='Category', how='left')
                                .merge(remm_com_sj_df, on='Category', how='left')
                                .merge(remm_other_sj_df, on='Category', how='left')
)


nbhd_df_merge3['NON_PARCEL_ACRES'] = round(nbhd_df_merge3['ACRES'] - nbhd_df_merge3['TTL_PARCEL_ACRES'],2)

nbhd_df_merge3.head()

,Category,SHAPE,DEVACRES,ACRES,MILES_ARTERIAL,MILES_COLLECTOR,MILES_LOCAL,MILES_ANY_ROAD,SF_UNITS,AVG_SF_FT2,AVG_SF_ACRES,TH_UNITS,AVG_TH_FT2,MF_UNITS,AVG_MF_FT2,TTL_PARCEL_ACRES,REMM_SF_UNITS,REMM_AVG_SF_FT2,SF_PARCEL_ACRES,REMM_AVG_SF_ACRES,REMM_MF_UNITS,MF_PARCEL_ACRES,REMM_AVG_MF_FT2,COM_PARCEL_ACRES,OTHR_PARCEL_ACRES,NON_PARCEL_ACRES
0,9th and 9th,"{'rings': [[[426491.5, 4512071.1], [426549.5, ...",391.556647,391.556647,0.51901,0.0,12.812107,13.331117,1275.0,2007.230591,0.136204,16.0,2019.776733,1013.0,1030.005127,272.152771,1295.0,2015.102661,176.913452,0.136541,1056.0,47.369434,1128.856689,12.348680,23.927864,119.40
1,"Avenues, Lower","{'rings': [[[425840.5, 4513912.4], [425916.200...",60.511321,60.511321,0.00000,0.0,2.408423,2.408423,88.0,2657.136475,0.128182,NaN,NaN,537.0,882.633118,37.007576,88.0,2657.136475,11.252505,0.128182,537.0,11.683568,882.633118,7.772047,6.145143,23.50
2,"Avenues, Upper","{'rings': [[[425951.53000000026, 4514917.67], ...",78.696582,78.696582,0.00000,0.0,3.233425,3.233425,311.0,2292.061035,0.126881,NaN,NaN,141.0,1211.045898,48.452084,319.0,2297.454590,40.370518,0.127147,141.0,7.175409,1211.045898,NaN,0.732411,30.24
3,"Daybreak, High Den","{'rings': [[[415038.4000000004, 4488766.800000...",210.999718,210.999718,0.00000,0.0,10.890334,10.890334,277.0,3109.864502,0.153529,294.0,1540.099609,385.0,1231.502441,152.135803,278.0,3109.864502,44.393219,0.152571,616.0,43.283825,1522.416870,9.920579,6.840915,58.86
4,"Daybreak, Low Den","{'rings': [[[415799.4000000004, 4490379.4], [4...",391.942869,391.942869,0.00000,0.0,18.340754,18.340754,1023.0,3491.598145,0.165827,244.0,2448.204834,NaN,NaN,293.793091,1026.0,3512.422607,170.658813,0.167077,274.0,18.022488,2384.278809,1.861134,96.413895,98.15


### SE Input

In [310]:
#========================
# 2019 SE
#========================

# t = taz_900.merge(se_2019, left_on='SA_TAZID', right_on=';TAZID', how='left')[['SA_TAZID', 'HHPOP', 'TOTHH', 'TOTEMP','RETEMP','INDEMP','OTHEMP', 'SHAPE']].copy()
# t.spatial.to_featureclass(location=os.path.join('.\inputs\se_2019.shp'),sanitize_columns=False)  

target_features = nbhd 
join_features = se_2019
output_features = os.path.join(gdb, "_08_se_2019")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

fields = ['HHPOP', 'TOTHH', 'TOTEMP','RETEMP','INDEMP','OTHEMP']
for f in fields:
    
    # sum se
    fieldindex = fieldmappings.findFieldMapIndex(f)
    fieldmap = fieldmappings.getFieldMap(fieldindex)
    fieldmap.mergeRule = 'Sum'
    fieldmappings.replaceFieldMap(fieldindex, fieldmap)


se_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="CONTAINS")

se_sj_df = pd.DataFrame.spatial.from_featureclass(se_sj[0])


In [311]:
se_sj_df= se_sj_df[['Category','HHPOP', 'TOTHH', 'TOTEMP','RETEMP','INDEMP','OTHEMP']].copy()
se_sj_df.rename({'OTHEMP':'OFFEMP'}, axis=1, inplace=True)
# merge the new data
nbhd_df_merge4 = nbhd_df_merge3.merge(se_sj_df, on='Category', how='left')

nbhd_df_merge4.head()

,Category,SHAPE,DEVACRES,ACRES,MILES_ARTERIAL,MILES_COLLECTOR,MILES_LOCAL,MILES_ANY_ROAD,SF_UNITS,AVG_SF_FT2,AVG_SF_ACRES,TH_UNITS,AVG_TH_FT2,MF_UNITS,AVG_MF_FT2,TTL_PARCEL_ACRES,REMM_SF_UNITS,REMM_AVG_SF_FT2,SF_PARCEL_ACRES,REMM_AVG_SF_ACRES,REMM_MF_UNITS,MF_PARCEL_ACRES,REMM_AVG_MF_FT2,COM_PARCEL_ACRES,OTHR_PARCEL_ACRES,NON_PARCEL_ACRES,HHPOP,TOTHH,TOTEMP,RETEMP,INDEMP,OFFEMP
0,9th and 9th,"{'rings': [[[426491.5, 4512071.1], [426549.5, ...",391.556647,391.556647,0.51901,0.0,12.812107,13.331117,1275.0,2007.230591,0.136204,16.0,2019.776733,1013.0,1030.005127,272.152771,1295.0,2015.102661,176.913452,0.136541,1056.0,47.369434,1128.856689,12.348680,23.927864,119.40,4855.567951,2288.000000,1370.666667,661.333333,33.0,676.333333
1,"Avenues, Lower","{'rings': [[[425840.5, 4513912.4], [425916.200...",60.511321,60.511321,0.00000,0.0,2.408423,2.408423,88.0,2657.136475,0.128182,NaN,NaN,537.0,882.633118,37.007576,88.0,2657.136475,11.252505,0.128182,537.0,11.683568,882.633118,7.772047,6.145143,23.50,937.879098,584.166667,644.000000,121.000000,7.0,516.000000
2,"Avenues, Upper","{'rings': [[[425951.53000000026, 4514917.67], ...",78.696582,78.696582,0.00000,0.0,3.233425,3.233425,311.0,2292.061035,0.126881,NaN,NaN,141.0,1211.045898,48.452084,319.0,2297.454590,40.370518,0.127147,141.0,7.175409,1211.045898,NaN,0.732411,30.24,958.002184,455.333333,0.000000,0.000000,0.0,0.000000
3,"Daybreak, High Den","{'rings': [[[415038.4000000004, 4488766.800000...",210.999718,210.999718,0.00000,0.0,10.890334,10.890334,277.0,3109.864502,0.153529,294.0,1540.099609,385.0,1231.502441,152.135803,278.0,3109.864502,44.393219,0.152571,616.0,43.283825,1522.416870,9.920579,6.840915,58.86,3000.139760,855.166667,3083.166667,97.000000,12.0,2974.166667
4,"Daybreak, Low Den","{'rings': [[[415799.4000000004, 4490379.4], [4...",391.942869,391.942869,0.00000,0.0,18.340754,18.340754,1023.0,3491.598145,0.165827,244.0,2448.204834,NaN,NaN,293.793091,1026.0,3512.422607,170.658813,0.167077,274.0,18.022488,2384.278809,1.861134,96.413895,98.15,4457.123481,1248.333333,228.000000,14.000000,56.0,158.000000


### Export

In [312]:
nbhd_df_merge4 = nbhd_df_merge4[['Category', 'SHAPE', 'DEVACRES', 'ACRES', 'TTL_PARCEL_ACRES', 'NON_PARCEL_ACRES', 'MILES_ARTERIAL',
       'MILES_COLLECTOR', 'MILES_LOCAL', 'MILES_ANY_ROAD', 'SF_UNITS',
       'AVG_SF_FT2', 'AVG_SF_ACRES', 'TH_UNITS', 'AVG_TH_FT2', 'MF_UNITS',
       'AVG_MF_FT2', 'REMM_SF_UNITS', 'REMM_AVG_SF_FT2',
       'REMM_AVG_SF_ACRES', 'REMM_MF_UNITS',
       'REMM_AVG_MF_FT2', 'SF_PARCEL_ACRES', 'MF_PARCEL_ACRES', 'COM_PARCEL_ACRES',
       'OTHR_PARCEL_ACRES', 'HHPOP', 'TOTHH', 'TOTEMP',
       'RETEMP', 'INDEMP', 'OFFEMP']]
nbhd_df_merge4.spatial.to_featureclass(location=os.path.join(gdb2, "TAZ_Grouping_Analysis"),sanitize_columns=False)

'e:\\Tasks\\MB-Neighborhood-Analysis\\Outputs\\TAZ_Grouping_Analysis.gdb\\TAZ_Grouping_Analysis'

In [313]:
del nbhd_df_merge4['SHAPE']
nbhd_df_merge4.to_csv(os.path.join(outputs[0], "TAZ_Grouping_Analysis.csv"))